# Compare results using various automatic metrics
In the translations, punctuation is separated by space from preceding words, so Levenstein distance is not completely correct but the results between different translations should be comparable most of the time.

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from os.path import join, splitext
import hlepor
import nltk
from razdel import tokenize

[nltk_data] Downloading package punkt to /home/denis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
path = 'Results'
filename = 'results.xlsx'

In [3]:
df = pd.read_excel(join(path, filename)).drop(columns='en')
ref = df.ru
df.drop(columns='ru', inplace=True)
df

,01_1_block_4_heads_128,02_2_blocks_4heads_128,03_3blocks_8heads_256,04_4blocks_8heads_512,05_8blocks_8heads_128
0,"я не знал , что ты имеешь в виду .","скажи им , что я тебе сказал .","скажи им то , что я тебе сказал .","скажи им , что я тебе сказал .","скажи им то , что я тебе сказал ."
1,я не хочу отмечать день рождения .,я не хочу отмечать свой год .,в этом году я не хочу отмечать день рождения .,я не хочу в этом году отмечать день рождения .,в этом году я не хочу отмечать день рождения .
2,"том не помнит , где припарковал машину .","том не мог вспомнить , где припарковал машину .","том не мог вспомнить , где припарковал машину .","том не мог вспомнить , где припарковал машину .","том не мог вспомнить , где припарковал машину ."
3,"том не хочет , чтобы я это сделал это сделал э...","том не сказал мне , чтобы я это говорил .","том не первый , кто мне это говорил .",том не был мне первым человеком .,"том не тот человек , который мне это было в пе..."
4,"учитель сказал мне , что учитель .","учитель сказал мне , чтобы я перестал стоять .",учитель сказал мне встать .,"учитель сказал мне , чтобы я встал .",учитель сказал мне встать .
...,...,...,...,...,...
95,"я хотел , чтобы том пошёл туда одной .",мне надо пойти встретиться с томом .,мне надо пойти встретить тома .,мне надо пойти встретиться с томом .,мне надо пойти встретить тома .
96,том очень хорошо поёт .,том очень хорошо это сделал .,том очень хорошо в этом поднаторел .,том в этом очень поднаторел .,том в этом очень поднаторел .
97,вы можете её расшифровать ?,когда ты пользуешься это ?,когда ты ей пользуешься ?,когда ты им пользуешься ?,когда ты ей пользуешься ?
98,это очень большая .,она большая .,оно большое .,он большой .,оно большое .


In [4]:
ref_tokenized = ref.map(lambda x: [token.text.lower() for token in tokenize(x)]).to_list()

In [5]:
def my_hlepor(hypoth, ref):
    params = {
        'alpha': 1.0,
        'beta': 1.0,
        'weight_elp': 2.0,
        'weight_pr': 3.0,
        'weight_pos': 1.0
    }
    return hlepor.hlepor_score(
        reference=ref,
        hypothesis=hypoth,
        **params)

In [6]:
def levenstein(hypoth, ref, substitution_cost=1, transpositions=False):
    def lev(row):
        return nltk.edit_distance(row['hypoth'], row['ref'], substitution_cost=substitution_cost, transpositions=transpositions)
    distances = pd.DataFrame({'hypoth': hypoth, 'ref': ref}).apply(lev, axis=1).mean()
        
    return distances

In [7]:
def chrF(hypoth, ref):
    return nltk.chrf_score.corpus_chrf(references=ref, hypotheses=hypoth, min_len=1, max_len=6, beta=3.0, ignore_whitespace=True)

In [8]:
result = df.apply(my_hlepor, axis=0, ref=ref).to_frame(name='hlepor') \
    .join(
        df.apply(levenstein, axis=0, ref=ref).to_frame(name='levenstein')
    ) \
    .join(
        df.apply(chrF, axis=0, ref=ref).to_frame(name='chrF')
    )
result

,hlepor,levenstein,chrF
01_1_block_4_heads_128,0.499352,39.70,0.252110
02_2_blocks_4heads_128,0.702994,15.85,0.458700
03_3blocks_8heads_256,0.764139,13.20,0.527081
04_4blocks_8heads_512,0.774775,12.69,0.559977
05_8blocks_8heads_128,0.766068,13.09,0.536985


In [9]:
result.to_excel(
    join(
        path,
        splitext(filename)[0] + '_metrics' + '.xlsx'
    )
)